# Capstone Project - The Battle of Neighborhoods

## Business Problem Section

#### Background

London is the largest city and the capital of United Kingdom. It is considered as one of the most important global cities. Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the third most populous of any city in Europe and accounts for 13.4% of the UK population. The city covers a total area of 607 sq. miles. Clearly, London is a city with high population density. Real estate investment in such cities thus becomes a big investment. And before making such an invetment, one should have a proper investment strategy.

#### Business Problem

In a country with such a huge area and high real estate prices, it becomes very tough to find a property with its appropriate value, keeping in ming all the amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores. As a result, the business problem we are currently posing is: How could we recommend profitable venues to support homebuyers according to amenities and essential facilities surrounding such venues?

## Data Section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/).
The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name, typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.